In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD, Adam
from keras import backend as K
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
#import PIL
from PIL import Image
import pandas as pd
import random
from sklearn.metrics import hamming_loss, precision_score, recall_score, f1_score
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
np.random.seed()

In [3]:
data = pd.read_csv('id_genre_pairs.csv')
movie_ids = list(data['movie_id'])
data = np.array(data)



In [4]:
import os
files = [i for i in os.listdir('Movie Posters Resized 32x32') if i[-3:] == 'jpg']
random.shuffle(files)

training_indices = files[:int(len(files)*0.8)]
test_indices = files[int(len(files)*0.8):]

test = []
test_labels = []
for i in test_indices:
    try:
        img = load_img('Movie Posters Resized 32x32/'+i)
        test.append(img_to_array(img))
        idx = int(i[:i.index(".")])
        for row in data:
            if row[0] == idx:
                test_labels.append(row[1:])
                break
    except:
        pass
    

In [5]:
train = []
train_labels = []
for i in training_indices:
    #try:
    img = load_img('Movie Posters Resized 32x32/'+i)
    train.append(img_to_array(img))
    idx = int(i[:i.index(".")])
    for row in data:
        if row[0] == idx:
            train_labels.append(row[1:])
            break
    #except:
        #pass



In [6]:
img_rows, img_cols = 32, 32
num_classes = 17

x_train = np.array(train) 
x_test = np.array(test)

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

In [7]:
y_train = np.array(train_labels)
y_test = np.array(test_labels)

# interestingly the keras example code does not center the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')



x_train shape: (11757, 32, 32, 3)
x_test shape: (2940, 32, 32, 3)
11757 train samples
2940 test samples


# Define Custom Loss Metric

In [58]:
from functools import partial
from itertools import product


def weighted_binary_crossentropy(y_true, y_pred, weights):
    nb_cl = len(weights)
    final_mask = K.zeros_like(y_pred[:, 0])
    y_pred_max = K.max(y_pred, axis=1)
    y_pred_max = K.expand_dims(y_pred_max, 1)
    y_pred_max_mat = K.equal(y_pred, y_pred_max)
    for c_p, c_t in product(range(nb_cl), range(nb_cl)):
        final_mask += (K.cast(weights[c_t, c_p],K.floatx()) * K.cast(y_pred_max_mat[:, c_p] ,K.floatx())* K.cast(y_true[:, c_t],K.floatx()))
    return K.mean(K.binary_crossentropy(y_pred, y_true), axis=-1)* final_mask

    
w_array = np.ones((17,17))
for i in range(17):
    w_array[9, i] = 10

ncce = partial(weighted_binary_crossentropy, weights=np.ones((17,17)))
ncce.__name__ ='weighted_binary_crossentropy'



In [16]:
from sklearn.metrics import hamming_loss, precision_score, recall_score, f1_score

In [59]:
import keras.backend as K
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dropout

def accuracy_with_threshold(y_true, y_pred):
    threshold = 0.1
    y_pred = K.cast(K.greater(y_pred, threshold), K.floatx())
    return K.mean(K.equal(y_true, y_pred))

from keras import regularizers

# smaller batch size means noisier gradient, but more updates per epoch
#batch_size = 256
batch_size=512 # Just for running locally - change this!
# number of iterations over the complete training data
epochs = 20
#epochs=1 # Just for testing - change this!
precisions = []
recalls = []
f1s = []
parameters = []
model_histories = []
# CV for number of layers, number of dense layers, sgd vs. adam, regularization constant, and learning rate 
for n_conv_layers in [1]:
    for n_dense_layers in [1]:
        for lrate in [0.001]:
            for adaptive_change in [0.01]:
                log_dir_string = "cv"
                log_dir_string += str(lrate)
                log_dir_string += str(adaptive_change)
                K.clear_session()
                model = Sequential()
                model.add(Conv2D(16, kernel_size=(5, 5), 
                                 activation='relu', 
                                 kernel_regularizer=regularizers.l2(0.01),
                                 input_shape=input_shape))
                model.add(MaxPooling2D(pool_size=(2, 2)))

                if n_conv_layers == 1:
                    log_dir_string += "_1conv"
                elif n_conv_layers==2:
                    model.add(Conv2D(16, kernel_size=(5, 5), 
                                     activation='relu', 
                                     kernel_regularizer=regularizers.l2(0.01),
                                     input_shape=input_shape))
                    model.add(MaxPooling2D(pool_size=(2, 2)))
                    log_dir_string += "_2conv"
                else:
                    model.add(Conv2D(16, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
                    model.add(MaxPooling2D(pool_size=(2, 2)))
                    log_dir_string += "_3conv"
                model.add(Flatten())
                model.add(Dense(64, activation='relu'))
                if n_dense_layers == 1:
                    log_dir_string += "_1fc"
                else:
                    model.add(Dense(64, activation='relu'))
                    log_dir_string += "_2fc"

                model.add(Dense(num_classes, activation='sigmoid'))

                adam = Adam(lr=lrate)
                model.compile(loss=ncce,
                      optimizer=adam,
                      metrics=['accuracy',accuracy_with_threshold])

                # we need a callback to save information for tensorboard visualizations
                tensorboard = TensorBoard(log_dir='./logs/cv/'+log_dir_string, histogram_freq=1, write_graph=True, write_images=False)

                reduce_lr = ReduceLROnPlateau(monitor='val_accuracy_with_threshold', factor = adaptive_change, patience = 5, min_lr = 0.00001)

                history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=(x_test, y_test),
                            callbacks=[tensorboard, reduce_lr])



                score = model.evaluate(x_test, y_test, verbose=0)
                predictions = (model.predict(x_test)>0.1).astype(int)
                print(n_conv_layers, n_dense_layers, lrate)
                parameters.append([n_conv_layers, n_dense_layers, lrate])
               
                model_histories.append(history)
                precision = precision_score(y_test, predictions, average="micro")
                recall = recall_score(y_test, predictions, average="micro")
                f1 = f1_score(y_test, predictions, average="micro")
                recalls.append(recall)
                precisions.append(precision)
                f1s.append(f1)
                
                print('Test loss:', score[0])
                print('Test accuracy:', score[1])
                print("Precision: ", precision, "micro, ", precision_score(y_test, predictions, average="macro"), " macro", )
                print("Recall: ", recall, "micro, ", recall_score(y_test, predictions, average="macro"), "macro ") 
                print("F1-score:", f1_score(y_test, predictions, average="micro"), "micro, ", f1_score(y_test, predictions, average="macro"), "macro")
                
            

                 

Train on 11757 samples, validate on 2940 samples
INFO:tensorflow:Summary name conv2d_1/kernel:0 is illegal; using conv2d_1/kernel_0 instead.
INFO:tensorflow:Summary name conv2d_1/bias:0 is illegal; using conv2d_1/bias_0 instead.
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
INFO:tensorflow:Summary name dense_2/kernel:0 is illegal; using dense_2/kernel_0 instead.
INFO:tensorflow:Summary name dense_2/bias:0 is illegal; using dense_2/bias_0 instead.
Epoch 1/20
11757/11757 [==============================] - 12s - loss: 0.5623 - acc: 0.2152 - accuracy_with_threshold: 0.5285 - val_loss: 0.4785 - val_acc: 0.2939 - val_accuracy_with_threshold: 0.6806
Epoch 2/20
11757/11757 [==============================] - 11s - loss: 0.4733 - acc: 0.2829 - accuracy_with_threshold: 0.6508 - val_loss: 0.4647 - val_acc: 0.2935 - val_accuracy_with_threshold: 0.6873
Epoch 3/20
11757/11

/Users/courtneycochrane/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/courtneycochrane/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
